In [89]:
import os
import shutil
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
# import cv2
import stable_whisper
from moviepy.editor import VideoFileClip
import csv
import torch
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.model_selection import train_test_split
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
from sentence_transformers import SentenceTransformer

In [5]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('punkt_tab')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /home/user1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/user1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/user1/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
# !sudo apt-get install ffmpeg -y

# Data Preprocessing

In [4]:
annotation = pickle.load(open('/home/user1/environments/train_dataset_vprod_encr_train/train/annotation/annotation_training.pkl', 'rb'), encoding='latin1')

FileNotFoundError: [Errno 2] No such file or directory: '/home/user1/environments/train_dataset_vprod_encr_train/train/annotation/annotation_training.pkl'

In [8]:
transcription = pickle.load(open('/home/user1/environments/train_dataset_vprod_encr_train/train/transcription/transcription_training.pkl', 'rb'), encoding='latin1')

In [9]:
annotation.keys()

dict_keys(['extraversion', 'neuroticism', 'agreeableness', 'conscientiousness', 'interview', 'openness'])

In [10]:
extraversion = []
neuroticism = []
agreeableness = []
conscientiousness = []
interview = []
openness = []
transcription_df = []

for video in annotation['extraversion']:
    extraversion.append({
        "video": video,
        "OCEAN": annotation['extraversion'][video]
    })

for video in annotation['neuroticism']:
    neuroticism.append({
        "video": video,
        "OCEAN": annotation['neuroticism'][video]
    })

for video in annotation['agreeableness']:
    agreeableness.append({
        "video": video,
        "OCEAN": annotation['agreeableness'][video]
    })

for video in annotation['conscientiousness']:
    conscientiousness.append({
        "video": video,
        "OCEAN": annotation['conscientiousness'][video]
    })

for video in annotation['interview']:
    interview.append({
        "video": video,
        "OCEAN": annotation['interview'][video]
    })

for video in annotation['openness']:
    openness.append({
        "video": video,
        "OCEAN": annotation['openness'][video]
    })

for video in transcription:
    transcription_df.append({
        "video": video,
        "text": transcription[video]
    })

extraversion = pd.DataFrame(extraversion)
neuroticism = pd.DataFrame(neuroticism)
agreeableness = pd.DataFrame(agreeableness)
conscientiousness = pd.DataFrame(conscientiousness)
interview = pd.DataFrame(interview)
openness = pd.DataFrame(openness)
transcription = pd.DataFrame(transcription_df)

In [11]:
data = []

for i in tqdm(range(len(agreeableness))):
    a = extraversion.iloc[i].OCEAN
    b = neuroticism.iloc[i].OCEAN
    c = agreeableness.iloc[i].OCEAN
    d = interview.iloc[i].OCEAN
    e = openness.iloc[i].OCEAN
    f = conscientiousness.iloc[i].OCEAN
    g = transcription.iloc[i]

    data.append({
        "video": extraversion.iloc[i].video,
        "text": g.text,
        "extraversion": a,
        "neuroticism": b,
        "agreeableness": c,
        "interview": d,
        "openness": e,
        "conscientiousness": f
    })

data = pd.DataFrame(data)

100%|████████████████████████████████████████████████| 6000/6000 [00:01<00:00, 3689.95it/s]


In [15]:
data.to_csv('data.csv', index=False)

In [ ]:
if not os.path.exists('/home/user1/environments/train_dataset_vprod_encr_train/train/data/videos'):
    os.mkdir('/home/user1/environments/train_dataset_vprod_encr_train/train/data/videos')

for folder in tqdm(os.listdir('/home/user1/environments/train_dataset_vprod_encr_train/train/data')):
    for video in os.listdir('/home/user1/environments/train_dataset_vprod_encr_train/train/data/' + folder):
        try:
            shutil.copy2('/home/user1/environments/train_dataset_vprod_encr_train/train/data/' + folder + '/' + video, '/home/user1/environments/train_dataset_vprod_encr_train/train/data/videos/' + video)
        except:
            pass

 13%|███████                                               | 10/76 [00:13<01:19,  1.20s/it]

 16%|████████▌                                             | 12/76 [00:16<01:19,  1.24s/it]

# Valid preprocessing

In [16]:
annotation = pickle.load(open('/home/user1/environments/train_dataset_vprod_encr_train/valid/annotation/annotation_validation.pkl', 'rb'), encoding='latin1')

In [17]:
transcription = pickle.load(open('/home/user1/environments/train_dataset_vprod_encr_train/valid/transcription/transcription_validation.pkl', 'rb'), encoding='latin1')

In [18]:
annotation.keys()

dict_keys(['extraversion', 'neuroticism', 'agreeableness', 'conscientiousness', 'interview', 'openness'])

In [19]:
extraversion = []
neuroticism = []
agreeableness = []
conscientiousness = []
interview = []
openness = []
transcription_df = []

for video in annotation['extraversion']:
    extraversion.append({
        "video": video,
        "OCEAN": annotation['extraversion'][video]
    })

for video in annotation['neuroticism']:
    neuroticism.append({
        "video": video,
        "OCEAN": annotation['neuroticism'][video]
    })

for video in annotation['agreeableness']:
    agreeableness.append({
        "video": video,
        "OCEAN": annotation['agreeableness'][video]
    })

for video in annotation['conscientiousness']:
    conscientiousness.append({
        "video": video,
        "OCEAN": annotation['conscientiousness'][video]
    })

for video in annotation['interview']:
    interview.append({
        "video": video,
        "OCEAN": annotation['interview'][video]
    })

for video in annotation['openness']:
    openness.append({
        "video": video,
        "OCEAN": annotation['openness'][video]
    })

for video in transcription:
    transcription_df.append({
        "video": video,
        "text": transcription[video]
    })

extraversion = pd.DataFrame(extraversion)
neuroticism = pd.DataFrame(neuroticism)
agreeableness = pd.DataFrame(agreeableness)
conscientiousness = pd.DataFrame(conscientiousness)
interview = pd.DataFrame(interview)
openness = pd.DataFrame(openness)
transcription = pd.DataFrame(transcription_df)

In [20]:
data_valid = []

for i in tqdm(range(len(agreeableness))):
    a = extraversion.iloc[i].OCEAN
    b = neuroticism.iloc[i].OCEAN
    c = agreeableness.iloc[i].OCEAN
    d = interview.iloc[i].OCEAN
    e = openness.iloc[i].OCEAN
    f = conscientiousness.iloc[i].OCEAN
    g = transcription.iloc[i]

    data_valid.append({
        "video": extraversion.iloc[i].video,
        "text": g.text,
        "extraversion": a,
        "neuroticism": b,
        "agreeableness": c,
        "interview": d,
        "openness": e,
        "conscientiousness": f
    })

data_valid = pd.DataFrame(data_valid)

100%|████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 3666.12it/s]


In [21]:
data_valid.to_csv('data_valid.csv', index=False)

In [42]:
if not os.path.exists('/home/user1/environments/train_dataset_vprod_encr_train/valid/data_valid/videos'):
    os.mkdir('/home/user1/environments/train_dataset_vprod_encr_train/valid/data_valid/videos')

for folder in tqdm(os.listdir('/home/user1/environments/train_dataset_vprod_encr_train/valid/data_valid')):
    for video in os.listdir('/home/user1/environments/train_dataset_vprod_encr_train/valid/data_valid/' + folder):
        try:
            shutil.copy2('/home/user1/environments/train_dataset_vprod_encr_train/valid/data_valid/' + folder + '/' + video, '/home/user1/environments/train_dataset_vprod_encr_train/valid/data_valid/videos/' + video)
        except:
            pass

100%|██████████████████████████████████████████████████████| 26/26 [00:35<00:00,  1.35s/it]


In [22]:
if not os.path.exists('/home/user1/environments/train_dataset_vprod_encr_train/videos'):
    os.mkdir('/home/user1/environments/train_dataset_vprod_encr_train/videos')

for folder in tqdm(os.listdir('/home/user1/environments/train_dataset_vprod_encr_train/valid/data_valid')):
    for video in os.listdir('/home/user1/environments/train_dataset_vprod_encr_train/valid/data_valid/' + folder):
        try:
            shutil.copy2('/home/user1/environments/train_dataset_vprod_encr_train/valid/data_valid/' + folder + '/' + video, '/home/user1/environments/train_dataset_vprod_encr_train/videos/' + video)
        except:
            pass

100%|██████████████████████████████████████████████████████| 26/26 [00:34<00:00,  1.32s/it]


In [23]:
for folder in tqdm(os.listdir('/home/user1/environments/train_dataset_vprod_encr_train/train/data')):
    for video in os.listdir('/home/user1/environments/train_dataset_vprod_encr_train/train/data/' + folder):
        try:
            shutil.copy2('/home/user1/environments/train_dataset_vprod_encr_train/train/data/' + folder + '/' + video, '/home/user1/environments/train_dataset_vprod_encr_train/videos/' + video)
        except:
            pass

100%|██████████████████████████████████████████████████████| 76/76 [00:46<00:00,  1.62it/s]


In [28]:
for i in tqdm(os.listdir('/home/user1/environments/train_dataset_vprod_encr_train/train/data/videos/')):
    shutil.move('/home/user1/environments/train_dataset_vprod_encr_train/train/data/videos/' + i, '/home/user1/environments/train_dataset_vprod_encr_train/videos/' + i)

100%|███████████████████████████████████████████████| 6000/6000 [00:00<00:00, 44994.41it/s]


In [29]:
for i in tqdm(os.listdir('/home/user1/environments/train_dataset_vprod_encr_train/valid/data_valid/videos/')):
    shutil.move('/home/user1/environments/train_dataset_vprod_encr_train/valid/data_valid/videos/' + i, '/home/user1/environments/train_dataset_vprod_encr_train/videos/' + i)

100%|███████████████████████████████████████████████| 1560/1560 [00:00<00:00, 50544.71it/s]


In [30]:
len(os.listdir('/home/user1/environments/train_dataset_vprod_encr_train/videos/'))

7560

# Data Analys

In [9]:
video_path = '/home/user1/environments/train_dataset_vprod_encr_train/train/data/videos/'

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
data['word_counts'] = data.text.apply(lambda x: len(x))

In [12]:
data.to_csv('data.csv', index=False)

In [13]:
data[['extraversion', 'neuroticism', 'agreeableness', 'interview', 'openness', 'conscientiousness']].corr()

,extraversion,neuroticism,agreeableness,interview,openness,conscientiousness
extraversion,1.000000,0.796407,0.710505,0.818464,0.774249,0.588592
neuroticism,0.796407,1.000000,0.732660,0.879114,0.774219,0.719671
agreeableness,0.710505,0.732660,1.000000,0.828069,0.649029,0.657955
interview,0.818464,0.879114,0.828069,1.000000,0.772283,0.832406
openness,0.774249,0.774219,0.649029,0.772283,1.000000,0.582695
conscientiousness,0.588592,0.719671,0.657955,0.832406,0.582695,1.000000


In [14]:
a = []

for i, row in tqdm(data.iterrows()):
    a.append((row.extraversion + row.neuroticism + row.agreeableness + row.openness + row.conscientiousness) / 5)

6000it [00:00, 14349.59it/s]


In [15]:
data['mean'] = a

In [16]:
data[['interview', 'mean']].corr()

,interview,mean
interview,1.000000,0.948591
mean,0.948591,1.000000


In [17]:
root_mean_squared_error(data.interview, data['mean'])

0.054275185794705104

In [18]:
X = data[['extraversion', 'neuroticism', 'agreeableness', 'openness', 'conscientiousness']]
y = data['interview']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [20]:
model = CatBoostRegressor(1000, verbose=False)
model.fit(X_train, y_train);

In [21]:
root_mean_squared_error(model.predict(X_test), y_test)

0.04356567549003082

In [12]:
data = pd.read_csv('environments/guivans_folder/data-2 (1).csv')
data

,video,text,extraversion,neuroticism,agreeableness,interview,openness,conscientiousness
0,J4GQm9j0JZ0.003.mp4,He's cutting it and then turn around and see t...,0.523364,0.552083,0.626374,0.504673,0.488889,0.601942
1,zEyRyTnIw5I.005.mp4,Responsibility to house the organ I had been g...,0.345794,0.375000,0.472527,0.457944,0.366667,0.582524
2,nskJh7v6v1U.004.mp4,I actually got quite a few sets of black pens ...,0.252336,0.291667,0.406593,0.373832,0.511111,0.485437
3,6wHQsN5g2RM.000.mp4,I ate a lot. I'd like a lot of foods. I rememb...,0.457944,0.489583,0.505495,0.457944,0.377778,0.398058
4,dQOeQYWIgm8.000.mp4,Now I'll ask you guys to leave a question in t...,0.607477,0.489583,0.406593,0.570093,0.622222,0.621359
...,...,...,...,...,...,...,...,...
5995,Eh7WRYXVh9M.000.mp4,"I got to see a lot, got to experience a lot. Y...",0.523364,0.479167,0.626374,0.588785,0.544444,0.621359
5996,2q8orkMs2Jg.003.mp4,"Really, really love my job, love the people I ...",0.728972,0.760417,0.582418,0.616822,0.822222,0.524272
5997,F1lAPYh4t3U.000.mp4,I have a nine-to-five. I said this in my last ...,0.700935,0.770833,0.747253,0.691589,0.788889,0.699029
5998,cxJ0u6r0-pU.001.mp4,You want to be working with and who you want t...,0.317757,0.531250,0.582418,0.616822,0.588889,0.679612


In [49]:
data['text'] = data['text'].astype(str)
data = data.dropna(subset=['text', 'extraversion'])

In [50]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    
    return text

In [51]:
data['text_preprocessed'] = data['text'].apply(preprocess_text)

In [52]:
data

,video,text,extraversion,neuroticism,agreeableness,interview,openness,conscientiousness,text_preprocessed
0,J4GQm9j0JZ0.003.mp4,He's cutting it and then turn around and see t...,0.523364,0.552083,0.626374,0.504673,0.488889,0.601942,hes cutting turn around see end result im glad...
1,zEyRyTnIw5I.005.mp4,Responsibility to house the organ I had been g...,0.345794,0.375000,0.472527,0.457944,0.366667,0.582524,responsibility house organ given needed tell g...
2,nskJh7v6v1U.004.mp4,I actually got quite a few sets of black pens ...,0.252336,0.291667,0.406593,0.373832,0.511111,0.485437,actually got quite sets black pens year bought...
3,6wHQsN5g2RM.000.mp4,I ate a lot. I'd like a lot of foods. I rememb...,0.457944,0.489583,0.505495,0.457944,0.377778,0.398058,ate lot id like lot foods remember favorite ma...
4,dQOeQYWIgm8.000.mp4,Now I'll ask you guys to leave a question in t...,0.607477,0.489583,0.406593,0.570093,0.622222,0.621359,ill ask guys leave question comments hopefully...
...,...,...,...,...,...,...,...,...,...
5995,Eh7WRYXVh9M.000.mp4,"I got to see a lot, got to experience a lot. Y...",0.523364,0.479167,0.626374,0.588785,0.544444,0.621359,got see lot got experience lot yeah didnt know...
5996,2q8orkMs2Jg.003.mp4,"Really, really love my job, love the people I ...",0.728972,0.760417,0.582418,0.616822,0.822222,0.524272,really really love job love people work feel f...
5997,F1lAPYh4t3U.000.mp4,I have a nine-to-five. I said this in my last ...,0.700935,0.770833,0.747253,0.691589,0.788889,0.699029,ninetofive said last qa ninetofive per se prob...
5998,cxJ0u6r0-pU.001.mp4,You want to be working with and who you want t...,0.317757,0.531250,0.582418,0.616822,0.588889,0.679612,want working want help bring anyone could get ...


In [28]:
def generate_additional_text_features(text):
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    num_words = len(words)
    num_sentences = len(sentences)
    avg_sentence_length = num_words / num_sentences if num_sentences > 0 else 0
    num_stopwords = sum(1 for word in words if word.lower() in stop_words)
    stopwords_ratio = num_stopwords / num_words if num_words > 0 else 0
    
    emotional_words = ["happy", "sad", "angry", "fear", "love", "hate", "joy", "surprise", "trust", "anticipation"]
    emotional_counts = Counter(word.lower() for word in words if word.lower() in emotional_words)
    emotional_frequency = {f"emotion_{emotion}": count / num_words for emotion, count in emotional_counts.items()}
    
    num_long_words = sum(1 for word in words if len(word) > 6)
    long_words_ratio = num_long_words / num_words if num_words > 0 else 0

    features = {
        "num_words": num_words,
        "num_sentences": num_sentences,
        "avg_sentence_length": avg_sentence_length,
        "stopwords_ratio": stopwords_ratio,
        "long_words_ratio": long_words_ratio,
        **emotional_frequency
    }
    
    return features

In [29]:
df = []

for text in data.text:
    df.append(generate_additional_text_features(str(text)))

df = pd.DataFrame(df)

In [30]:
data = pd.concat([data, df], axis=1).fillna(0)

In [31]:
X = data.drop(columns=['video', 'extraversion', 'neuroticism', 'agreeableness', 'interview', 'openness', 'conscientiousness', 'text', 'text_preprocessed'])
y = data.extraversion

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2)

In [33]:
model = CatBoostRegressor(use_best_model=True, verbose=100, eval_metric='MAE', ) #text_features=['text', 'text_preprocessed']
model.fit(X_train, y_train, eval_set=(X_test, y_test))

Learning rate set to 0.06514
0:	learn: 0.1235878	test: 0.1239416	best: 0.1239416 (0)	total: 1.52ms	remaining: 1.52s
100:	learn: 0.1204136	test: 0.1247796	best: 0.1239416 (0)	total: 104ms	remaining: 924ms
200:	learn: 0.1173594	test: 0.1254273	best: 0.1239416 (0)	total: 206ms	remaining: 820ms
300:	learn: 0.1141545	test: 0.1260420	best: 0.1239416 (0)	total: 308ms	remaining: 716ms
400:	learn: 0.1112353	test: 0.1264748	best: 0.1239416 (0)	total: 409ms	remaining: 611ms
500:	learn: 0.1086278	test: 0.1267984	best: 0.1239416 (0)	total: 510ms	remaining: 508ms
600:	learn: 0.1060774	test: 0.1271906	best: 0.1239416 (0)	total: 613ms	remaining: 407ms
700:	learn: 0.1034435	test: 0.1275572	best: 0.1239416 (0)	total: 717ms	remaining: 306ms
800:	learn: 0.1013336	test: 0.1279253	best: 0.1239416 (0)	total: 820ms	remaining: 204ms
900:	learn: 0.0993126	test: 0.1283292	best: 0.1239416 (0)	total: 922ms	remaining: 101ms
999:	learn: 0.0973198	test: 0.1285444	best: 0.1239416 (0)	total: 1.02s	remaining: 0us

bestT

In [7]:
from transformers import BertTokenizer, BertForSequenceClassification

In [8]:
tokenizer = BertTokenizer.from_pretrained("Minej/bert-base-personality")
model = BertForSequenceClassification.from_pretrained("Minej/bert-base-personality")

def personality_detection(text):
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    outputs = model(**inputs)
    predictions = outputs.logits.squeeze().detach().numpy()

    label_names = ['Extroversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']
    result = {label_names[i]: predictions[i] for i in range(len(label_names))}

    return result

tokenizer_config.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [13]:
data

,video,text,extraversion,neuroticism,agreeableness,interview,openness,conscientiousness
0,J4GQm9j0JZ0.003.mp4,He's cutting it and then turn around and see t...,0.523364,0.552083,0.626374,0.504673,0.488889,0.601942
1,zEyRyTnIw5I.005.mp4,Responsibility to house the organ I had been g...,0.345794,0.375000,0.472527,0.457944,0.366667,0.582524
2,nskJh7v6v1U.004.mp4,I actually got quite a few sets of black pens ...,0.252336,0.291667,0.406593,0.373832,0.511111,0.485437
3,6wHQsN5g2RM.000.mp4,I ate a lot. I'd like a lot of foods. I rememb...,0.457944,0.489583,0.505495,0.457944,0.377778,0.398058
4,dQOeQYWIgm8.000.mp4,Now I'll ask you guys to leave a question in t...,0.607477,0.489583,0.406593,0.570093,0.622222,0.621359
...,...,...,...,...,...,...,...,...
5995,Eh7WRYXVh9M.000.mp4,"I got to see a lot, got to experience a lot. Y...",0.523364,0.479167,0.626374,0.588785,0.544444,0.621359
5996,2q8orkMs2Jg.003.mp4,"Really, really love my job, love the people I ...",0.728972,0.760417,0.582418,0.616822,0.822222,0.524272
5997,F1lAPYh4t3U.000.mp4,I have a nine-to-five. I said this in my last ...,0.700935,0.770833,0.747253,0.691589,0.788889,0.699029
5998,cxJ0u6r0-pU.001.mp4,You want to be working with and who you want t...,0.317757,0.531250,0.582418,0.616822,0.588889,0.679612


In [20]:
personality_detection(data.iloc[0].text)

{'Extroversion': 0.17577949,
 'Neuroticism': 0.06724694,
 'Agreeableness': 0.0672438,
 'Conscientiousness': -0.19891754,
 'Openness': 0.16153476}

In [53]:
abc = []

for i, row in tqdm(data.iterrows()):
    text = str(row.text_preprocessed)
    abc.append(personality_detection(text))

abc = pd.DataFrame(abc)
abc

6000it [06:19, 15.80it/s]


,Extroversion,Neuroticism,Agreeableness,Conscientiousness,Openness
0,0.050920,0.098305,-0.169253,-0.792664,0.013037
1,0.163965,0.155553,-0.375368,-0.926911,0.062853
2,0.064799,0.110624,-0.334377,-0.896726,0.060532
3,0.057381,0.218640,-0.186665,-0.905684,0.140462
4,0.249427,0.076483,-0.373975,-1.020600,0.065929
...,...,...,...,...,...
5995,0.130956,0.145444,-0.204086,-0.823804,0.035300
5996,0.288234,0.238590,-0.443498,-1.129111,0.188047
5997,-0.003428,0.019143,-0.134987,-0.817869,-0.248194
5998,0.098568,0.243888,-0.359588,-1.018936,0.093485


In [55]:
def map_range(x, a, b, c, d):
    return c + (d - c) * (x - a) / (b - a)

In [56]:
a = abc.copy()

In [57]:
a.Extroversion.max()

0.51976234

In [58]:
abc.Extroversion.min()

-0.79762787

In [59]:
root_mean_squared_error(data.extraversion, abc.Extroversion)

0.4345540965545271

In [60]:
root_mean_squared_error(data.extraversion, a.Extroversion.map(lambda x: map_range(x, -0.79762787, 0.51976234, 0, 1)))

0.2921321674228767

In [63]:
root_mean_squared_error(data.neuroticism, abc.Neuroticism)

0.41769940055031707

In [64]:
root_mean_squared_error(data.agreeableness, abc.Agreeableness)

0.8789985047270198

In [65]:
root_mean_squared_error(data.openness, abc.Openness)

0.5157688823139074

In [66]:
root_mean_squared_error(data.conscientiousness, abc.Conscientiousness)

1.480150620192249

In [77]:
X = data[['extraversion']]
y = data.conscientiousness

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=42, test_size=0.15)

In [79]:
model = CatBoostRegressor(verbose=False, eval_metric='RMSE')
model.fit(X_train, y_train)

In [80]:
root_mean_squared_error(y_test, model.predict(X_test))

0.12190148463295875

In [83]:
data_valid = pd.read_csv('environments/data_valid.csv')
data_valid

,video,text,extraversion,neuroticism,agreeableness,interview,openness,conscientiousness
0,modNfUPt3F4.002.mp4,[inaudible 00:00:16] and then I will do the vo...,0.644860,0.593750,0.615385,0.616822,0.555556,0.640777
1,h6LOjpCRXtY.005.mp4,"Personality is A1, I like you. So that is all ...",0.439252,0.520833,0.417582,0.439252,0.411111,0.572816
2,WER4ww680QQ.004.mp4,"Ewe. But yeah, some people been asking me if I...",0.457944,0.312500,0.428571,0.373832,0.555556,0.398058
3,c4XnKouozXU.002.mp4,"I am a makeup artist, I am an aesthetician, I ...",0.364486,0.572917,0.527473,0.523364,0.322222,0.553398
4,OEKg-Tvwcbk.002.mp4,"Sucks. When he agrees, yeah. It just sucks bec...",0.345794,0.468750,0.516484,0.383178,0.477778,0.417476
...,...,...,...,...,...,...,...,...
1995,3LAaFUSGvsU.000.mp4,Really qualified to. The guy who last week was...,0.570093,0.614583,0.494505,0.626168,0.577778,0.689320
1996,n2BuwHbdilY.000.mp4,"Wearing scarfs on the second date. Okay, so no...",0.542056,0.541667,0.549451,0.579439,0.666667,0.669903
1997,GcuoyJPO-KU.003.mp4,"""How do you manage to do things with someone? ...",0.551402,0.593750,0.560440,0.504673,0.644444,0.572816
1998,uf_sIIw4zxY.004.mp4,"-I sell. No. Sophie Loveland, ""Will you be pos...",0.514019,0.552083,0.461538,0.551402,0.733333,0.572816


In [84]:
data_valid['text_preprocessed'] = data_valid['text'].apply(preprocess_text)

In [87]:
data

,video,text,extraversion,neuroticism,agreeableness,interview,openness,conscientiousness,text_preprocessed
0,J4GQm9j0JZ0.003.mp4,He's cutting it and then turn around and see t...,0.523364,0.552083,0.626374,0.504673,0.488889,0.601942,hes cutting turn around see end result im glad...
1,zEyRyTnIw5I.005.mp4,Responsibility to house the organ I had been g...,0.345794,0.375000,0.472527,0.457944,0.366667,0.582524,responsibility house organ given needed tell g...
2,nskJh7v6v1U.004.mp4,I actually got quite a few sets of black pens ...,0.252336,0.291667,0.406593,0.373832,0.511111,0.485437,actually got quite sets black pens year bought...
3,6wHQsN5g2RM.000.mp4,I ate a lot. I'd like a lot of foods. I rememb...,0.457944,0.489583,0.505495,0.457944,0.377778,0.398058,ate lot id like lot foods remember favorite ma...
4,dQOeQYWIgm8.000.mp4,Now I'll ask you guys to leave a question in t...,0.607477,0.489583,0.406593,0.570093,0.622222,0.621359,ill ask guys leave question comments hopefully...
...,...,...,...,...,...,...,...,...,...
5995,Eh7WRYXVh9M.000.mp4,"I got to see a lot, got to experience a lot. Y...",0.523364,0.479167,0.626374,0.588785,0.544444,0.621359,got see lot got experience lot yeah didnt know...
5996,2q8orkMs2Jg.003.mp4,"Really, really love my job, love the people I ...",0.728972,0.760417,0.582418,0.616822,0.822222,0.524272,really really love job love people work feel f...
5997,F1lAPYh4t3U.000.mp4,I have a nine-to-five. I said this in my last ...,0.700935,0.770833,0.747253,0.691589,0.788889,0.699029,ninetofive said last qa ninetofive per se prob...
5998,cxJ0u6r0-pU.001.mp4,You want to be working with and who you want t...,0.317757,0.531250,0.582418,0.616822,0.588889,0.679612,want working want help bring anyone could get ...


In [4]:
!wget https://lodmedia.hb.bizmrg.com/case_files/1179238/train_dataset_vprod_encr_train.zip

--2024-11-09 11:02:15--  https://lodmedia.hb.bizmrg.com/case_files/1179238/train_dataset_vprod_encr_train.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 95.163.53.117
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|95.163.53.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23525252079 (22G) [application/zip]
Saving to: ‘train_dataset_vprod_encr_train.zip’

train_dataset_vprod   2%[                    ] 568.27M  3.97MB/s    eta 97m 45s^C


In [3]:
!rm train_dataset_vprod_encr_train.zip

In [ ]:
!ls

: 